In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

City_ID        City Country                 Date      Lat       Lng  \
0        0  Bredasdorp      ZA  2021-10-02 15:39:10 -34.5322   20.0403   
1        1         Mao      ES  2021-10-02 15:36:46  39.8885    4.2658   
2        2     Aksarka      RU  2021-10-02 15:39:11  66.5606   67.7975   
3        3    Nirasaki      JP  2021-10-02 15:39:11  35.7000  138.4500   
4        4    Katangli      RU  2021-10-02 15:39:12  51.7102  143.2326   

   Max Temp  Humidity  Cloudiness  Wind Speed  
0     56.28        50          90       13.80  
1     73.96        68          40        8.05  
2     31.01        81          76       11.43  
3     61.05        94           1        2.86  
4     42.98        70           6        8.88

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# 1. Assign the locations to an array of latitude and longitude pairs.
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
# Assign the figure variable.
fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# Assign the heatmap variable.
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# 5. Add the heatmap layetr.
# Add the heatmap layer.
fig.add_layer(heatmap_layer)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# 6. Call the figure to plot the data.
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [8]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

City_ID                          City Country                 Date  \
1         1                           Mao      ES  2021-10-02 15:36:46   
6         6                         Chama      CH  2021-10-02 15:39:12   
7         7                       Rikitea      PF  2021-10-02 15:39:12   
11       11            Puerto Del Rosario      ES  2021-10-02 15:39:14   
13       13                     Shelburne      CA  2021-10-02 15:39:14   
14       14                        Destin      US  2021-10-02 15:34:30   
15       15                     Jamestown      US  2021-10-02 15:38:48   
16       16  Presidencia Roque Saenz Pena      AR  2021-10-02 15:39:15   
17       17                    Phetchabun      TH  2021-10-02 15:39:15   
21       21                Ribeira Grande      PT  2021-10-02 15:39:16   

        Lat       Lng  Max Temp  Humidity  Cloudiness  Wind Speed  
1   39.8885    4.2658     73.96        68          40        8.05  
6   47.1821    8.4636     71.76        56          70        2.30  
7  -23.1203 -134.9692     72.64        72           2       21.25  
11  28.5004  -13.8627     83.88        78           0       17.27  
13  44.0787  -80.2041     70.12        61          99       13.67  
14  30.3935  -86.4958     84.40        82           1        9.22  
15  42.0970  -79.2353     68.02        64           1        4.00  
16 -26.7852  -60.4388     72.36        73          99        6.08  
17  16.2500  101.0833     77.95        89          98        2.08  
21  38.5167  -28.7000     77.40        78          40       14.97

In [12]:
preferred_cities_df.count()

City_ID       300
City          300
Country       299
Date          300
Lat           300
Lng           300
Max Temp      300
Humidity      300
Cloudiness    300
Wind Speed    300
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp      Lat       Lng  \
1                            Mao      ES     73.96  39.8885    4.2658   
6                          Chama      CH     71.76  47.1821    8.4636   
7                        Rikitea      PF     72.64 -23.1203 -134.9692   
11            Puerto Del Rosario      ES     83.88  28.5004  -13.8627   
13                     Shelburne      CA     70.12  44.0787  -80.2041   
14                        Destin      US     84.40  30.3935  -86.4958   
15                     Jamestown      US     68.02  42.0970  -79.2353   
16  Presidencia Roque Saenz Pena      AR     72.36 -26.7852  -60.4388   
17                    Phetchabun      TH     77.95  16.2500  101.0833   
21                Ribeira Grande      PT     77.40  38.5167  -28.7000   

   Hotel Name  
1              
6              
7              
11             
13             
14             
15             
16             
17             
21

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uECfbdp8pieUuXgP_4gxhNFQw_ZmxstiJ2ZvhMd4MSSP8J21vVA8JoWMQ7xpCJKra5qSRqI3CpBFcPU7SywI7kjXJ3_26Ey0tcKjdhaVT1JbJDILIdSLAc3uhzuJ-MbRCBg2os9w4FFgP5z7mC2pWfd-EH7gDpZjwAkSLKBXR2X5vr7co-nO6Y7nlNjueInW2ZlGKUedH3DRHU09MWy8FxnUQgXe2luNWZLMLirTdxPOpIGg2G7wsE4n1TOJyYCqcVOIGH2Baa6bI3jWXg1Fpby2xEEKm8wHlxQfNYF1jVeycQYg2d_t5i7CxP7cDh8WlgMFCD-1JnMu5dGVGY2QqoNtloLeavTVW064zDb5ROVDBLaIBG3okjWlmMdSFLoW0qrlj99vEIZalH0Mq4RFf1Qc844jLjZ1y0Tuve1EJXbfJaPMzPZrxWHf',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    ## Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

City Country  Max Temp      Lat       Lng  \
1                   Mao      ES     73.96  39.8885    4.2658   
6                 Chama      CH     71.76  47.1821    8.4636   
7               Rikitea      PF     72.64 -23.1203 -134.9692   
11   Puerto Del Rosario      ES     83.88  28.5004  -13.8627   
13            Shelburne      CA     70.12  44.0787  -80.2041   
..                  ...     ...       ...      ...       ...   
673         San Nicolas      PH     78.71  18.1725  120.5953   
675             Manggar      ID     79.12  -2.8833  108.2667   
677             Camacha      PT     72.36  33.0833  -16.3333   
678             Bergama      TR     76.23  39.1207   27.1805   
685              Jishou      CN     78.46  28.3167  109.7167   

                      Hotel Name  
1                   ARTIEM Capri  
6              City Garden Hotel  
7                People ThankYou  
11   Hotel JM Puerto del Rosario  
13       The Retreat at the Farm  
..                           ...  
673              NORTHVIEW Hotel  
675                  Guest Hotel  
677      Hotel Porto Santo & Spa  
678                   Anil Hotel  
685                   7 Days Inn  

[300 rows x 6 columns]

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [21]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))